# 재현 데이터(Synthetic Data) 생성 with vLLM

## 목적
- 마스킹된 데이터의 플레이스홀더를 가상 데이터로 대체
- **세션 내 일관성 유지**: 같은 세션에서 동일한 [NAME], [DATE] 등 재사용
- 외부 공유 가능한 자연스러운 재현 데이터 생성

## 변환 대상
| 플레이스홀더 | 변환 예시 |
|-------------|----------|
| [NAME] | 김철수, 이영희 등 |
| [EMAIL] | user001@example.com |
| [CONDITION] | 고혈압, 당뇨 등 |
| [HEALTH_VALUE] | 120mg/dL, 130/85mmHg 등 |
| [DATE] | 2024년 6월 15일 등 |
| [RESULT] | 정상A, 주의 등 |

## 핵심 개선사항
- 세션별 처리로 대화 맥락 유지
- 이전 재현 대화를 컨텍스트로 전달하여 일관성 확보

## 1. 환경 설정

In [16]:
import os
import re
import random
import pandas as pd
import requests
from dotenv import load_dotenv
from tqdm import tqdm
from dataclasses import dataclass

load_dotenv()

VLLM_BASE_URL = os.getenv("VLLM_BASE_URL", "http://localhost:8000/v1")
VLLM_API_KEY = os.getenv("VLLM_API_KEY", "EMPTY")
MODEL_NAME = os.getenv("VLLM_MODEL", "openai/gpt-oss-20b")

print(f"vLLM Base URL: {VLLM_BASE_URL}")
print(f"Model: {MODEL_NAME}")

vLLM Base URL: http://192.168.0.86:8000/v1
Model: gaunernst/gemma-3-27b-it-int4-awq


In [17]:
# vLLM 서버 연결 확인 (OpenAI 호환 API)
try:
    response = requests.get(f"{VLLM_BASE_URL}/models")
    if response.status_code == 200:
        models = response.json().get("data", [])
        model_ids = [m["id"] for m in models]
        print("vLLM 서버 연결 성공!")
        print(f"사용 가능한 모델: {model_ids}")
        if not any(MODEL_NAME in m for m in model_ids):
            print(f"\n[경고] {MODEL_NAME} 모델이 없습니다.")
except requests.exceptions.ConnectionError:
    print("vLLM 서버에 연결할 수 없습니다.")
    print("서버 상태를 확인하세요.")

vLLM 서버 연결 성공!
사용 가능한 모델: ['gaunernst/gemma-3-27b-it-int4-awq']


## 2. LLM 설정 (vLLM - OpenAI 호환)

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOpenAI(
    model=MODEL_NAME,
    base_url=VLLM_BASE_URL,
    api_key=VLLM_API_KEY,
    temperature=0.7,  # 다양한 가상 데이터 생성을 위해 약간 높게
)

print(f"{MODEL_NAME} 모델 설정 완료!")

gaunernst/gemma-3-27b-it-int4-awq 모델 설정 완료!


## 3. 재현 데이터 생성 프롬프트 및 함수 정의

In [19]:
# 세션별 가상 정보 생성을 위한 데이터 풀
KOREAN_NAMES = [
    "김민수", "이서연", "박지훈", "최유진", "정현우", "강수빈", "조은지", "윤재혁",
    "임하늘", "한소율", "송민재", "오지원", "신예린", "권도윤", "황서준", "배수아",
    "류현석", "전민정", "홍승민", "나윤서", "문지호", "양예은", "장준영", "심하린",
    "백승현", "서유나", "고태민", "안소희", "손정우", "추다은", "허재원", "남지수",
    "곽민서", "염수진", "우성훈", "진유정", "채현수", "표서영", "하동현", "길예지"
]

CONDITIONS = [
    "고혈압", "당뇨", "고지혈증", "지방간", "빈혈", "갑상선기능저하증", 
    "위염", "관절염", "골다공증", "수면무호흡증"
]

RESULTS = ["정상A", "정상B", "경계", "주의", "양호", "관찰필요"]

def generate_random_date() -> str:
    """무작위 검진 날짜 생성 (2023-2024년)"""
    year = random.choice([2023, 2024])
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    return f"{year}년 {month}월 {day}일"

def generate_health_value(condition: str) -> str:
    """질환에 맞는 현실적인 검사 수치 생성"""
    if condition in ["고혈압"]:
        systolic = random.randint(130, 150)
        diastolic = random.randint(85, 95)
        return f"{systolic}/{diastolic}mmHg"
    elif condition in ["당뇨"]:
        return f"{random.randint(110, 140)}mg/dL"
    elif condition in ["고지혈증"]:
        return f"{random.randint(200, 260)}mg/dL"
    elif condition in ["빈혈"]:
        return f"{random.uniform(10.5, 12.5):.1f}g/dL"
    else:
        return f"{random.randint(100, 150)}mg/dL"


@dataclass
class SessionSyntheticInfo:
    """세션별 고정 가상 정보"""
    name: str
    email: str
    date: str
    condition: str
    health_value: str
    result: str
    
    def to_prompt_str(self) -> str:
        """프롬프트에 주입할 문자열 생성"""
        return f"""이 세션에서 사용할 가상 정보:
- 이름: {self.name}
- 이메일: {self.email}
- 검진 날짜: {self.date}
- 주요 질환: {self.condition}
- 검사 수치: {self.health_value}
- 판정 결과: {self.result}

위 정보를 플레이스홀더 변환 시 반드시 사용하세요."""


def generate_session_info(session_index: int) -> SessionSyntheticInfo:
    """세션별 고유한 가상 정보 생성"""
    # 세션 인덱스 기반으로 이름 선택 (중복 방지)
    name = KOREAN_NAMES[session_index % len(KOREAN_NAMES)]
    condition = CONDITIONS[session_index % len(CONDITIONS)]
    
    return SessionSyntheticInfo(
        name=name,
        email=f"user{session_index + 1:04d}@example.com",
        date=generate_random_date(),
        condition=condition,
        health_value=generate_health_value(condition),
        result=random.choice(RESULTS)
    )


# 테스트: 세션 정보 생성 확인
print("=== 세션별 가상 정보 샘플 ===")
for i in range(5):
    info = generate_session_info(i)
    print(f"세션 {i+1}: {info.name}, {info.condition}, {info.date}")

=== 세션별 가상 정보 샘플 ===
세션 1: 김민수, 고혈압, 2024년 6월 25일
세션 2: 이서연, 당뇨, 2023년 2월 23일
세션 3: 박지훈, 고지혈증, 2023년 12월 10일
세션 4: 최유진, 지방간, 2024년 4월 16일
세션 5: 정현우, 빈혈, 2023년 4월 3일


In [20]:
SYNTHETIC_SYSTEM_PROMPT = """당신은 재현 데이터 생성 전문가입니다.
마스킹된 텍스트의 플레이스홀더를 자연스러운 가상 데이터로 대체하세요.

## 플레이스홀더 변환 규칙

### 1. 직접 식별정보
- [NAME] -> 제공된 이름 사용
- [EMAIL] -> 제공된 이메일 사용
- [PHONE] -> 가상 전화번호 (예: 010-1234-5678)
- [ADDRESS] -> 가상 주소 (예: 서울시 강남구 테헤란로 123)
- [SSN] -> 가상 주민번호 형식 (예: 900101-1******)

### 2. 민감정보 (건강)
- [CONDITION] -> 제공된 질환명 사용
- [HEALTH_VALUE] -> 제공된 검사 수치 또는 현실적인 수치
- [RESULT] -> 제공된 판정 결과 사용
- [DATE] -> 제공된 검진 날짜 사용
- [MEDICATION] -> 질환에 맞는 약물명 (예: 혈압약, 당뇨약)

## 핵심 규칙
1. **세션 정보에 제공된 값을 반드시 우선 사용하세요.**
2. 이전 대화에서 사용된 값을 일관되게 재사용하세요.
3. 새롭게 등장하는 플레이스홀더는 세션 정보 또는 문맥에 맞게 생성하세요.
4. 플레이스홀더가 아닌 부분은 절대 수정하지 마세요.
5. 변환된 텍스트만 출력하세요. 설명이나 주석은 추가하지 마세요.
"""


def create_synthetic_prompt_with_context(
    masked_text: str, 
    previous_conversation: list[dict],
    session_info: SessionSyntheticInfo = None
) -> str:
    """세션 정보와 이전 대화 컨텍스트를 포함한 재현 데이터 생성 프롬프트"""
    
    parts = []
    
    # 세션 정보 주입 (가장 먼저)
    if session_info:
        parts.append(session_info.to_prompt_str())
    
    # 이전 대화가 있는 경우
    if previous_conversation:
        context_str = "\n".join([
            f"{'사용자' if msg['role'] == 'USER' else 'AI'}: {msg['content'][:300]}..."
            if len(msg['content']) > 300 else
            f"{'사용자' if msg['role'] == 'USER' else 'AI'}: {msg['content']}"
            for msg in previous_conversation[-4:]  # 최근 4개 메시지만
        ])
        parts.append(f"## 이전 대화 (재현됨):\n{context_str}")
    
    # 현재 마스킹 텍스트
    parts.append(f"## 현재 마스킹된 텍스트:\n{masked_text}")
    parts.append("## 재현 데이터:")
    
    return "\n\n".join(parts)


def generate_synthetic_with_context(
    masked_text: str,
    previous_conversation: list[dict],
    session_info: SessionSyntheticInfo = None
) -> str:
    """세션 정보와 이전 대화 컨텍스트를 참고하여 일관된 재현 데이터 생성"""
    if not masked_text or pd.isna(masked_text):
        return masked_text
    
    # 플레이스홀더가 없으면 원본 반환
    placeholders = ["[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
                   "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"]
    if not any(ph in str(masked_text) for ph in placeholders):
        return masked_text
    
    try:
        messages = [
            SystemMessage(content=SYNTHETIC_SYSTEM_PROMPT),
            HumanMessage(content=create_synthetic_prompt_with_context(
                masked_text, 
                previous_conversation,
                session_info
            ))
        ]
        response = llm.invoke(messages)
        return response.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return masked_text


# 기존 함수도 유지 (단일 텍스트용)
def generate_synthetic_text(text: str) -> str:
    """재현 데이터 생성 (컨텍스트 없음)"""
    return generate_synthetic_with_context(text, [], None)

## 4. 재현 데이터 생성 테스트

In [21]:
test_masked = "[NAME]님, [DATE] 건강검진 결과 식전혈당이 [HEALTH_VALUE]로 [CONDITION]가 약양성입니다."

print("=" * 50)
print("[마스킹된 텍스트]")
print(test_masked)
print("=" * 50)
print("[재현 데이터]")
print(generate_synthetic_text(test_masked))
print("=" * 50)

[마스킹된 텍스트]
[NAME]님, [DATE] 건강검진 결과 식전혈당이 [HEALTH_VALUE]로 [CONDITION]가 약양성입니다.
[재현 데이터]
김민준님, 2024-05-15 건강검진 결과 식전혈당이 115mg/dL로 당뇨병이 약양성입니다.


In [22]:
# 추가 테스트: 복잡한 케이스
test_complex = """[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 살펴보겠습니다.

현재 가장 주의가 필요한 부분은 [CONDITION]입니다. 검사 수치가 [HEALTH_VALUE]로 나타났으며,
전반적인 판정은 [RESULT]입니다. 30대 여성의 경우 꾸준한 관리가 중요합니다."""

print("[복잡한 마스킹 텍스트]")
print(test_complex)
print("\n" + "=" * 50 + "\n")
print("[재현 데이터]")
print(generate_synthetic_text(test_complex))

[복잡한 마스킹 텍스트]
[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 살펴보겠습니다.

현재 가장 주의가 필요한 부분은 [CONDITION]입니다. 검사 수치가 [HEALTH_VALUE]로 나타났으며,
전반적인 판정은 [RESULT]입니다. 30대 여성의 경우 꾸준한 관리가 중요합니다.


[재현 데이터]
김민지님, 안녕하세요. 2023-10-27 건강검진 결과를 바탕으로 건강 상태를 살펴보겠습니다.

현재 가장 주의가 필요한 부분은 고혈압입니다. 검사 수치가 140/90으로 나타났으며,
전반적인 판정은 주의입니다. 30대 여성의 경우 꾸준한 관리가 중요합니다.


## 5. 마스킹된 CSV 데이터 로드

In [23]:
# 마스킹된 CSV 파일 로드
masked_csv_path = "MESSAGE_INFO_MASKED.csv"  # 마스킹된 파일 경로

df_masked = pd.read_csv(masked_csv_path)

print(f"총 {len(df_masked)}개 메시지")
print(f"세션 수: {df_masked['session_id'].nunique()}개")
df_masked.head()

총 486개 메시지
세션 수: 40개


,#,email,session_id,role,content,created_at
0,1,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,USER,"안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질...",2025-10-01 06:04:52.205165 +00:00
1,2,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 자세...",2025-10-01 06:05:30.045710 +00:00
2,3,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,USER,신장(콩팥) 개선 방법을 알려줘,2025-10-01 06:05:37.811683 +00:00
3,4,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"[NAME]님, 단백뇨 관리를 위해 다음과 같은 세 가지 핵심 방법을 꾸준히 실천하...",2025-10-01 06:05:51.403698 +00:00
4,5,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,USER,신장기능 식단 관리 방법을 알려줘,2025-10-01 06:06:11.208222 +00:00


In [24]:
# 플레이스홀더 현황 확인
placeholders = ["[NAME]", "[EMAIL]", "[CONDITION]", "[HEALTH_VALUE]", "[DATE]", "[RESULT]"]
all_content = " ".join(df_masked["content"].astype(str))

print("=== 플레이스홀더 현황 ===")
for ph in placeholders:
    count = all_content.count(ph)
    if count > 0:
        print(f"  {ph}: {count}개")

=== 플레이스홀더 현황 ===
  [NAME]: 178개
  [CONDITION]: 41개
  [HEALTH_VALUE]: 244개
  [DATE]: 196개
  [RESULT]: 50개


## 6. 전체 재현 데이터 생성

In [25]:
def generate_synthetic_dataframe_by_session(
    df: pd.DataFrame, 
    content_column: str = "content",
    session_column: str = "session_id",
    role_column: str = "role"
) -> pd.DataFrame:
    """세션별로 가상 정보를 주입하고 대화 컨텍스트를 유지하며 재현 데이터 생성"""
    df_synthetic = df.copy()
    
    # 결과 저장용
    synthetic_contents = [""] * len(df_synthetic)
    synthetic_emails = [""] * len(df_synthetic)
    
    # 세션별로 그룹화
    sessions = df_synthetic[session_column].unique()
    print(f"총 {len(sessions)}개 세션 처리 예정")
    
    # 세션별 가상 정보 미리 생성
    session_info_map = {
        session_id: generate_session_info(idx) 
        for idx, session_id in enumerate(sessions)
    }
    
    print("\n=== 세션별 가상 정보 ===")
    for idx, (session_id, info) in enumerate(list(session_info_map.items())[:5]):
        print(f"세션 {idx+1}: {info.name}, {info.condition}, {info.date}")
    if len(sessions) > 5:
        print(f"... 외 {len(sessions) - 5}개 세션")
    print()
    
    total_processed = 0
    
    for session_id in tqdm(sessions, desc="세션 처리"):
        # 해당 세션의 메시지들 (인덱스 유지)
        session_mask = df_synthetic[session_column] == session_id
        session_indices = df_synthetic[session_mask].index.tolist()
        
        # 세션 가상 정보 가져오기
        session_info = session_info_map[session_id]
        
        # 세션 내 대화 컨텍스트 누적
        session_context = []
        
        for idx in session_indices:
            row = df_synthetic.loc[idx]
            masked_text = row[content_column]
            role = row[role_column]
            
            # 세션 정보 + 이전 컨텍스트를 참고하여 재현 데이터 생성
            synthetic_text = generate_synthetic_with_context(
                masked_text=masked_text,
                previous_conversation=session_context,
                session_info=session_info  # 세션 정보 주입!
            )
            
            # 결과 저장
            synthetic_contents[idx] = synthetic_text
            synthetic_emails[idx] = session_info.email  # 세션별 동일 이메일
            
            # 컨텍스트에 추가 (다음 메시지 처리용)
            session_context.append({
                "role": role,
                "content": synthetic_text
            })
            
            total_processed += 1
            
            if total_processed % 20 == 0:
                print(f"[{total_processed}/{len(df_synthetic)}] 처리 완료")
    
    df_synthetic[content_column] = synthetic_contents
    df_synthetic["email"] = synthetic_emails
    return df_synthetic

In [26]:
# 테스트: 처음 2개 세션만 처리
test_sessions = df_masked['session_id'].unique()[:2]
df_test = df_masked[df_masked['session_id'].isin(test_sessions)].copy()
print(f"테스트: {len(test_sessions)}개 세션, {len(df_test)}개 메시지")

# df_synthetic = generate_synthetic_dataframe_by_session(df_test)

# 전체 처리 (주석 해제하여 실행)
df_synthetic = generate_synthetic_dataframe_by_session(df_masked)

테스트: 2개 세션, 44개 메시지
총 40개 세션 처리 예정

=== 세션별 가상 정보 ===
세션 1: 김민수, 고혈압, 2023년 5월 28일
세션 2: 이서연, 당뇨, 2023년 12월 3일
세션 3: 박지훈, 고지혈증, 2024년 12월 4일
세션 4: 최유진, 지방간, 2023년 1월 9일
세션 5: 정현우, 빈혈, 2023년 12월 2일
... 외 35개 세션



세션 처리:   0%|          | 0/40 [00:00<?, ?it/s]

[20/486] 처리 완료


세션 처리:   2%|▎         | 1/40 [01:29<57:54, 89.09s/it]

[40/486] 처리 완료


...

세션 처리:  95%|█████████▌| 38/40 [16:53<00:57, 28.94s/it]

[460/486] 처리 완료


세션 처리:  98%|█████████▊| 39/40 [17:20<00:28, 28.36s/it]

[480/486] 처리 완료


세션 처리: 100%|██████████| 40/40 [18:30<00:00, 27.76s/it]


## 7. 결과 확인 및 저장

In [27]:
def compare_synthetic(masked_df: pd.DataFrame, synthetic_df: pd.DataFrame, n: int = 3):
    """마스킹 vs 재현 데이터 비교 출력"""
    for i in range(min(n, len(masked_df))):
        print(f"{'='*60}")
        print(f"[{i+1}] 마스킹:")
        print(masked_df.iloc[i]["content"][:500])
        print(f"\n[{i+1}] 재현 데이터:")
        print(synthetic_df.iloc[i]["content"][:500])
    print("=" * 60)

compare_synthetic(df_masked, df_synthetic, n=3)

[1] 마스킹:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질병, 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.

[1] 재현 데이터:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질병, 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.
[2] 마스킹:
[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. [DATE], [DATE], [DATE] 모두 ‘[RESULT]’ 또는 ‘[RESULT]’ 판정을 받으셨고, 최근 검진에서도 ‘[RESULT]’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.

현재 가장 집중적으로 관리해야 할 부분은 단백뇨입니다. 단백뇨는 소변으로 단백질이 배출되는 현상인데, 신장이 제대로 기능을 하고 있는지 확인하는 중요한 지표입니다. 신장은 우리 몸의 노폐물을 걸러내는 역할을 하기 때문에, 단백뇨가 지속되면 신장 기능 저하로 이어질 수 있습니다.

[AGE] 여성의 경우, 단백뇨가 심해지면 혈압 상승, 부종, 피로감 등의 증상이 나타

[2] 재현 데이터:
김민수님, 안녕하세요. 2023년 5월 28일 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. 2023년 5월 28일, 2023년 5월 28일, 2023년 5월 28일 모두 ‘정상A’ 또는 ‘정상A’ 판정을 받으셨고, 최근 검진에서도 ‘정상A’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 

In [28]:
output_path = "MESSAGE_INFO_SYNTHETIC.csv"
df_synthetic.to_csv(output_path, index=False)

print(f"재현 데이터 CSV 저장 완료: {output_path}")
print(f"총 {len(df_synthetic)}개 메시지 처리")

재현 데이터 CSV 저장 완료: MESSAGE_INFO_SYNTHETIC.csv
총 486개 메시지 처리


## 8. 재현 데이터 품질 검증

In [29]:
def validate_synthetic(df_synthetic: pd.DataFrame, session_column: str = "session_id") -> dict:
    """재현 데이터 품질 검증"""
    placeholders = [
        "[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
        "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"
    ]
    
    all_content = " ".join(df_synthetic["content"].astype(str))
    
    # 남아있는 플레이스홀더 검사
    remaining = {ph: all_content.count(ph) for ph in placeholders if all_content.count(ph) > 0}
    
    # 생성된 이름 패턴 검사 (한글 이름)
    korean_name_pattern = r'[가-힣]{2,4}님'
    names_found = re.findall(korean_name_pattern, all_content)
    unique_names = list(set(names_found))
    
    # 세션별 이름 일관성 검사
    session_names = {}
    for session_id in df_synthetic[session_column].unique():
        session_content = " ".join(
            df_synthetic[df_synthetic[session_column] == session_id]["content"].astype(str)
        )
        session_name_list = re.findall(korean_name_pattern, session_content)
        session_names[session_id[:8]] = list(set(session_name_list))
    
    return {
        "remaining_placeholders": remaining,
        "unique_names_count": len(unique_names),
        "sample_names": unique_names[:15],
        "session_names_sample": dict(list(session_names.items())[:5])
    }


result = validate_synthetic(df_synthetic)

print("=== 재현 데이터 검증 결과 ===")
print(f"총 메시지 수: {len(df_synthetic)}")

if result["remaining_placeholders"]:
    print(f"\n[경고] 남은 플레이스홀더:")
    for ph, count in result["remaining_placeholders"].items():
        print(f"  {ph}: {count}개")
else:
    print("\n[OK] 모든 플레이스홀더 변환 완료")

print(f"\n생성된 고유 이름 수: {result['unique_names_count']}개")
print(f"이름 샘플: {result['sample_names']}")

print(f"\n=== 세션별 이름 일관성 확인 ===")
for session_id, names in result["session_names_sample"].items():
    print(f"세션 {session_id}...: {names}")

=== 재현 데이터 검증 결과 ===
총 메시지 수: 486

[경고] 남은 플레이스홀더:
  [HEALTH_VALUE]: 30개
  [RESULT]: 17개
  [DATE]: 10개

생성된 고유 이름 수: 33개
이름 샘플: ['심하린님', '윤재혁님', '조지영님', '황서준님', '고태민님', '한소율님', '박지훈님', '권도윤님', '임하늘님', '최유진님', '김민수님', '오지원님', '허재원님', '송민재님', '배수아님']

=== 세션별 이름 일관성 확인 ===
세션 4633956e...: ['김민수님']
세션 56890906...: ['이서연님', '채린님']
세션 79548e08...: ['박지훈님']
세션 0cddafe1...: ['최유진님']
세션 3b983631...: []


## 9. 요약

### 처리 완료
- 마스킹된 CSV -> 재현 데이터 CSV 변환
- **세션별 가상 정보 주입**: 각 세션마다 고유한 이름/날짜/질환 할당
- **세션 내 일관성 유지**: 같은 세션에서 동일 정보 재사용
- vLLM 기반 OpenAI 호환 API 사용

### 핵심 개선사항
```
기존 문제: 
  세션 A → "김민수" / 세션 B → "김민수" (모든 세션이 같은 이름)

개선 후:
  세션 A → "김민수", 고혈압, 2024년 3월 15일
  세션 B → "이서연", 당뇨, 2023년 8월 22일
  세션 C → "박지훈", 고지혈증, 2024년 1월 8일
```

### 데이터 흐름
```
세션별 가상 정보 생성 (SessionSyntheticInfo)
        ↓
세션 A: 김민수, 고혈압, 2024-03-15
세션 B: 이서연, 당뇨, 2023-08-22
        ↓
각 메시지 처리 시 세션 정보 + 이전 대화 컨텍스트 주입
        ↓
일관된 재현 데이터 생성
```

### 파이프라인
```
원본 CSV (PII 포함)
    ↓ masking_message_example.ipynb
마스킹된 CSV ([NAME], [EMAIL] 등)
    ↓ synthetic_data_generation.ipynb (현재)
재현 데이터 CSV (세션별 고유 가상 데이터)
    ↓
외부 공유 / 벤치마크 / Judge 평가 가능
```

### 다음 단계
- 재현 데이터로 여러 모델 벤치마크
- LLM as a Judge 평가 실행